<a href="https://colab.research.google.com/github/rachidkarakhi/Arduino-RFID-RC522-VB-Net-Interface-MySQL-Database/blob/main/bert_multi_with_a_ra_q.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install nltk
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
print("GPU is available: ", tf.config.experimental.list_physical_devices("GPU"))

GPU is available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importing the dataset

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd

def get_data_from_xml_child(child):
    Row = {}
    Row["StudentID"] = child.find("MetaInfo").get('StudentID')
    Row["TaskID"] = child.find("MetaInfo").get('TaskID')
    Row["DataSource"] = child.find("MetaInfo").get('DataSource')
    Row["ProblemDescription"] = child.find("ProblemDescription").text
    Row["Question"] = child.find("Question").text
    Row["Answer"] = child.find("Answer").text
    Row["Annotation_label"] = child.find("Annotation").get("Label")
    Row["Annotation_ContextRequired"] = child.find("Annotation").find("AdditionalAnnotation").get('ContextRequired')
    Row["Annotation_ExtraInfoInAnswer"] = child.find("Annotation").find("AdditionalAnnotation").get('ExtraInfoInAnswer')
    Row["Annotation_comments"] = child.find("Annotation").find("Comments").text
    Row["Annotation_comments_watch"] = child.find("Annotation").find("Comments").get("Watch")
    Row["ReferenceAnswers"] = child.find("ReferenceAnswers").text
    return Row

# xml_data = open('/kaggle/input/asag-dt-grad-xml/grade_data.xml', 'r').read()  # Read file
# Read the file from your Google Drive
with open('/content/drive/MyDrive/Colab Notebooks/grade_data.xml', 'r') as f:
    xml_data = f.read()
root = ET.XML(xml_data)  # Parse XML

data = []
cols = []
for i, child in enumerate(root):
    data.append(get_data_from_xml_child(child))

In [ ]:
df = pd.DataFrame(data)
df.head()

,StudentID,TaskID,DataSource,ProblemDescription,Question,Answer,Annotation_label,Annotation_ContextRequired,Annotation_ExtraInfoInAnswer,Annotation_comments,Annotation_comments_watch,ReferenceAnswers
0,DTSU040,LP03_PR09.bLK.sh,DeepTutorSummer2014,"A car windshield collides with a mosquito, squ...",How does Newton's third law apply to this situ...,the windshield will apply a force to the mosqu...,correct(0)|correct_but_incomplete(1)|contradic...,0,0,The student forgot to tell the opposite force...,1,\n1: Since the windshield exerts a force on t...
1,DTSU035,FM_LV04_PR05.sh,DeepTutorSummer2014,Two hockey players pass a puck between them on...,What forces are acting on the puck while the p...,The normal force coming from the ice and the g...,correct(1)|correct_but_incomplete(0)|contradic...,0,1,Indirectly mentioned that the puck moves in a...,1,\n1: The forces acting on the puck while it i...
2,DTSU021,FM_LVxx_PR01,DeepTutorSummer2014,A rocket pushes a meteor with constant force. ...,Can you articulate Newton's second law?,"if there is a zero net force on the object, th...",correct(0)|correct_but_incomplete(0)|contradic...,0,0,"Is this the correct definition, not listed in ...",1,\n1: Newton's 2nd Law says that the net force...
3,DTSU033,LP03_PR09.bLK.sh,DeepTutorSummer2014,"A car windshield collides with a mosquito, squ...",Can you articulate a principle or definition w...,An equal force always balancing it out regardl...,correct(0)|correct_but_incomplete(0)|contradic...,0,0,So difficult to understand. Based on the refe...,1,"\n1: For every action, there is an equal and ..."
4,DTSU015,FM_LV04_PR05,DeepTutorSummer2014,Two hockey players pass a puck between them on...,"Based on Newton's first law, what can you say ...",The speed of the puck will equal to the net fo...,correct(0)|correct_but_incomplete(0)|contradic...,0,0,Related but not correct.,0,\n1: The puck will move in a straight line wi...


In [ ]:
df.columns

Index(['StudentID', 'TaskID', 'DataSource', 'ProblemDescription', 'Question',
       'Answer', 'Annotation_label', 'Annotation_ContextRequired',
       'Annotation_ExtraInfoInAnswer', 'Annotation_comments',
       'Annotation_comments_watch', 'ReferenceAnswers'],
      dtype='object')

In [ ]:
df_draft = df[['ProblemDescription', 'Question',
       'Answer', 'Annotation_ContextRequired', 'Annotation_ExtraInfoInAnswer',
       'Annotation_comments', 'Annotation_comments_watch', 'ReferenceAnswers',"Annotation_label"]]
df_draft.columns = ['PD', 'Q','A', 'ACR', 'AEI','AC', 'ACW', 'RA',"target"]

In [ ]:
df_draft.head()

,PD,Q,A,ACR,AEI,AC,ACW,RA,target
0,"A car windshield collides with a mosquito, squ...",How does Newton's third law apply to this situ...,the windshield will apply a force to the mosqu...,0,0,The student forgot to tell the opposite force...,1,\n1: Since the windshield exerts a force on t...,correct(0)|correct_but_incomplete(1)|contradic...
1,Two hockey players pass a puck between them on...,What forces are acting on the puck while the p...,The normal force coming from the ice and the g...,0,1,Indirectly mentioned that the puck moves in a...,1,\n1: The forces acting on the puck while it i...,correct(1)|correct_but_incomplete(0)|contradic...
2,A rocket pushes a meteor with constant force. ...,Can you articulate Newton's second law?,"if there is a zero net force on the object, th...",0,0,"Is this the correct definition, not listed in ...",1,\n1: Newton's 2nd Law says that the net force...,correct(0)|correct_but_incomplete(0)|contradic...
3,"A car windshield collides with a mosquito, squ...",Can you articulate a principle or definition w...,An equal force always balancing it out regardl...,0,0,So difficult to understand. Based on the refe...,1,"\n1: For every action, there is an equal and ...",correct(0)|correct_but_incomplete(0)|contradic...
4,Two hockey players pass a puck between them on...,"Based on Newton's first law, what can you say ...",The speed of the puck will equal to the net fo...,0,0,Related but not correct.,0,\n1: The puck will move in a straight line wi...,correct(0)|correct_but_incomplete(0)|contradic...


In [ ]:
def change_label(txt):
    rs = ""
    splits = str(txt).split("|")


    for sp in splits:
        #print(sp)
        if "1" in sp:
            #print("in")
            rs = rs+sp+" "

    return rs.replace("(1)","").strip()

In [ ]:
df_draft["target"] = df_draft["target"].apply(change_label)

In [ ]:
df_draft["target"].value_counts()

correct                             367
incorrect                           237
correct_but_incomplete              209
contradictory                        84
correct_but_incomplete incorrect      1
Name: target, dtype: int64

In [ ]:
df1 = df_draft[["A","RA","target","ACR","AEI","ACW"]]
df1.head()

,A,RA,target,ACR,AEI,ACW
0,the windshield will apply a force to the mosqu...,\n1: Since the windshield exerts a force on t...,correct_but_incomplete,0,0,1
1,The normal force coming from the ice and the g...,\n1: The forces acting on the puck while it i...,correct,0,1,1
2,"if there is a zero net force on the object, th...",\n1: Newton's 2nd Law says that the net force...,incorrect,0,0,1
3,An equal force always balancing it out regardl...,"\n1: For every action, there is an equal and ...",incorrect,0,0,1
4,The speed of the puck will equal to the net fo...,\n1: The puck will move in a straight line wi...,incorrect,0,0,0


In [ ]:
#  df_draft["text"] = df_draft["A"]+" "+df_draft["RA"]
# df_draft["text"] = df_draft["A"]+" "+df_draft["RA"]

In [ ]:
# df_draft["text"] = df_draft["Q"]+" "+df_darft["A"]+" "+df_darft["RA"]

In [ ]:
#$ df1 = df_draft[["text","target","ACR","AEI","ACW"]]

In [ ]:
# len(df1["text"][55])

In [ ]:
def transfrom_class(txt):
    txt = txt.strip()
    if "correct" == txt:
        return 0
    elif "incorrect" == txt:
        return 1
    elif "correct_but_incomplete" == txt:
        return 2
    elif "contradictory" == txt:
        return 3
    elif "correct_but_incomplete incorrect" == txt:
        return 4

In [ ]:
df1["target"].apply(transfrom_class).value_counts()

0    367
1    237
2    209
3     84
4      1
Name: target, dtype: int64

In [ ]:
df1["target"] = df1["target"].apply(transfrom_class)

In [ ]:
df1.head()

,A,RA,target,ACR,AEI,ACW
0,the windshield will apply a force to the mosqu...,\n1: Since the windshield exerts a force on t...,2,0,0,1
1,The normal force coming from the ice and the g...,\n1: The forces acting on the puck while it i...,0,0,1,1
2,"if there is a zero net force on the object, th...",\n1: Newton's 2nd Law says that the net force...,1,0,0,1
3,An equal force always balancing it out regardl...,"\n1: For every action, there is an equal and ...",1,0,0,1
4,The speed of the puck will equal to the net fo...,\n1: The puck will move in a straight line wi...,1,0,0,0


In [ ]:
df1 = df1.drop(df1[df1["target"] == 4].index)

In [ ]:
df1["target"].value_counts()

0    367
1    237
2    209
3     84
Name: target, dtype: int64

In [ ]:
import spacy
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import string

In [ ]:
nlp = spacy.load('en_core_web_sm')

stopwords = _stop_words.ENGLISH_STOP_WORDS
lemmatizer = WordNetLemmatizer()

def clean(doc):
    #text_no_namedentities = []
    #document = nlp(doc)
    #ents = [e.text for e in document.ents]
    #for item in document:
    #    if item.text in ents:
    #        pass
    #    else:
    #        text_no_namedentities.append(item.text)

    #doc = (" ".join(text_no_namedentities))

    doc = doc.lower().strip()
    doc = doc.replace("</br>", " ")
    doc = doc.replace("-", " ")

    doc = "".join([char for char in doc if char not in string.punctuation and not char.isdigit()])
    doc = " ".join([token for token in doc.split() if token not in stopwords])
    doc = "".join([lemmatizer.lemmatize(word) for word in doc])
    return doc

In [ ]:
df1.A.iloc[0]
df1.RA.iloc[0]


'\n1:  Since the windshield exerts a force on the mosquito, which we can call action, the mosquito exerts an equal and opposite force on the windshield, called the reaction.\n2:  The action  is the windshield squashing the mosquito, and the equal and opposite reaction is the mosquito hitting the windshield.\n3:  The force exerted by the windshield on the mosquito and the force exerted by the mosquito on the windshield are a third-law pair of action and reaction.\n4:  The force exerted by the windshield on the mosquito and the force exerted by the mosquito on the windshield are an action-reaction pair\n'

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
clean(df1.A.iloc[0])
clean(df1.RA.iloc[0])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


'windshield exerts force mosquito action mosquito exerts equal opposite force windshield called reaction action windshield squashing mosquito equal opposite reaction mosquito hitting windshield force exerted windshield mosquito force exerted mosquito windshield law pair action reaction force exerted windshield mosquito force exerted mosquito windshield action reaction pair'

In [ ]:
df1["A"] = df1.A.apply(clean)
df1["RA"] = df1.RA.apply(clean)

In [ ]:
df1.head()

,A,RA,target,ACR,AEI,ACW
0,windshield apply force mosquito equal force ap...,windshield exerts force mosquito action mosqui...,2,0,0,1
1,normal force coming ice gravitational force ve...,forces acting puck players downward force grav...,0,0,1,1
2,zero net force object velocity constant,newtons nd law says net force equal product ma...,1,0,0,1
3,equal force balancing regardless different mas...,action equal opposite reaction object exerts f...,1,0,0,1
4,speed puck equal net force times mass puck,puck straight line constant speed,1,0,0,0


In [ ]:
# df1 = pd.read_csv("/kaggle/input/asag-dt-grad/clean_data_4_labels.csv")

In [ ]:
print(df1.columns)
# del df1["Unnamed: 0"]

Index(['A', 'RA', 'target', 'ACR', 'AEI', 'ACW'], dtype='object')


In [ ]:
df1.head()

,A,RA,target,ACR,AEI,ACW
0,windshield apply force mosquito equal force ap...,windshield exerts force mosquito action mosqui...,2,0,0,1
1,normal force coming ice gravitational force ve...,forces acting puck players downward force grav...,0,0,1,1
2,zero net force object velocity constant,newtons nd law says net force equal product ma...,1,0,0,1
3,equal force balancing regardless different mas...,action equal opposite reaction object exerts f...,1,0,0,1
4,speed puck equal net force times mass puck,puck straight line constant speed,1,0,0,0


In [ ]:
df = df1[["A","RA","target"]]

In [ ]:
batch_1 = df

In [ ]:
batch_1["target"].value_counts()

0    367
1    237
2    209
3     84
Name: target, dtype: int64

In [ ]:
# For DistilBERT:
# model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
# model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# # Load pretrained model/tokenizer
# tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
# model = model_class.from_pretrained(pretrained_weights)

In [ ]:
# The Sentence Pair Classification task
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased')


# # Load the BERT Large tokenizer
# tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")

# # Load the BERT Large model
# model = BertForSequenceClassification.from_pretrained("bert-large-uncased")

# Define the RoBERTa model and tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base')

# Define the RoBERTa model and tokenizer
# tokenizer = RobertaTokenizer.from_pretrained('roberta-large')
# model = RobertaForSequenceClassification.from_pretrained('roberta-large')


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Set the number of labels to match the number of classes in your dataset
# Change this to match the number of classes in your dataset
num_labels = 4
model.config.num_labels = num_labels

Model #1: Preparing the Dataset

Right now, the variable `model` holds a pretrained distilBERT model -- a version of BERT that is smaller, but much faster and requiring a lot less memory.

## Model #1: Preparing the Dataset
Before we can hand our sentences to BERT, we need to do some minimal processing to put them in the format it requires.

### Tokenization
Our first step is to tokenize the sentences -- break them up into word and subwords in the format BERT is comfortable with.

In [ ]:
inputs = [(a, ra) for a, ra in zip(batch_1["A"], batch_1["RA"])]
# tokenizer(, , return_tensors='pt')
encoded_inputs = tokenizer(inputs,
                           padding=True,
                           truncation=True,
                           return_tensors='pt')
encoded_inputs

{'input_ids': tensor([[    0,  9428, 44656,  ...,     1,     1,     1],
        [    0, 21113,  1370,  ...,     1,     1,     1],
        [    0, 18556,  1161,  ...,     1,     1,     1],
        ...,
        [    0, 10845,  4289,  ...,     1,     1,     1],
        [    0, 40412,  1095,  ...,     1,     1,     1],
        [    0,  5646, 43746,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
# tokenized = batch_1["clean_text"].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
batch_1.reset_index(drop=True, inplace=True)
tokenized = []
for i in range(len(batch_1["A"])):
    # Tokenize the sentence pair
    encoded = tokenizer.encode_plus(batch_1["A"][i], batch_1["RA"][i],
                                     add_special_tokens=True,
                                     max_length=512,
                                     truncation=True,
                                     padding='max_length',
                                     return_attention_mask=True,
                                     return_token_type_ids=True)
    # Add the encoded sequence to the list
    tokenized.append(encoded)

In [ ]:
# max_len = 0
# for i in tokenized.values:
#     if len(i) > max_len:
#         max_len = len(i)
max_len = 0
for i in tokenized:
    if len(i) > max_len:
        max_len = len(i)
padded = np.array([enc['input_ids'] + [0]*(max_len-len(enc['input_ids'])) for enc in tokenized])



In [ ]:
padded

array([[    0,  9428, 44656, ...,     1,     1,     1],
       [    0, 21113,  1370, ...,     1,     1,     1],
       [    0, 18556,  1161, ...,     1,     1,     1],
       ...,
       [    0, 10845,  4289, ...,     1,     1,     1],
       [    0, 40412,  1095, ...,     1,     1,     1],
       [    0,  5646, 43746, ...,     1,     1,     1]])

In [ ]:
np.array(padded).shape

(897, 512)

### Masking
If we directly send `padded` to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(897, 512)

In [ ]:
# input_ids = torch.tensor(padded)
# attention_mask = torch.tensor(attention_mask)

# with torch.no_grad():
#     last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
# input_ids = torch.tensor(padded)
# attention_mask = torch.tensor(attention_mask)
# batch_size = 16
# num_samples = len(input_ids)
# last_hidden_states = []

# with torch.no_grad():
#     for i in range(0, num_samples, batch_size):
#         input_ids_batch = torch.tensor(padded[i:i+batch_size])
#         attention_mask_batch = torch.tensor(attention_mask[i:i+batch_size])
#         batch_last_hidden_states = model(input_ids_batch, attention_mask=attention_mask_batch)
#         last_hidden_states.append(batch_last_hidden_states.last_hidden_state)

# last_hidden_states = torch.cat(last_hidden_states, dim=0)
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)
batch_size = 16
num_samples = len(input_ids)
last_hidden_states = []

with torch.no_grad():
    for i in range(0, num_samples, batch_size):
        input_ids_batch = torch.tensor(padded[i:i+batch_size])
        attention_mask_batch = torch.tensor(attention_mask[i:i+batch_size])
        batch_last_hidden_states = model(input_ids_batch, attention_mask=attention_mask_batch)
        last_hidden_states.append(batch_last_hidden_states.logits)

last_hidden_states = torch.cat(last_hidden_states, dim=0)


In [ ]:
if len(last_hidden_states.shape) == 2:
    features = last_hidden_states.numpy()
else:
    features = last_hidden_states[:, 0, :].numpy()

#  features = last_hidden_states[:, 0, :].numpy()
#  features = last_hidden_states[:, :].numpy()
# last_hidden_states
len(last_hidden_states.shape)

2

In [ ]:
labels = batch_1["target"]
print('test print here ')

test print here 


In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)
train_features

array([[-0.05862097, -0.00657541],
       [-0.03356872, -0.00230301],
       [-0.02560998, -0.01544097],
       ...,
       [-0.04083653, -0.00784247],
       [-0.0134796 ,  0.00331267],
       [-0.04221137,  0.03259893]], dtype=float32)

We now train the LogisticRegression model. If you've chosen to do the gridsearch, you can plug the value of C into the model declaration (e.g. `LogisticRegression(C=5.2)`).


In [ ]:
from sklearn.naive_bayes import GaussianNB

from sklearn.tree import DecisionTreeClassifier

In [ ]:
lr_clf = LogisticRegression()
#lr_clf = DecisionTreeClassifier()
lr_clf.fit(train_features, train_labels)

LogisticRegression()

In [ ]:
ypred = lr_clf.predict(test_features)

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_score(ypred, test_labels,average='weighted')

0.538961038961039

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(test_labels, ypred))

              precision    recall  f1-score   support

           0       0.37      1.00      0.54        83
           1       0.00      0.00      0.00        65
           2       0.00      0.00      0.00        51
           3       0.00      0.00      0.00        26

    accuracy                           0.37       225
   macro avg       0.09      0.25      0.13       225
weighted avg       0.14      0.37      0.20       225



In [ ]:
dt_clf = DecisionTreeClassifier(max_depth=2)
dt_clf.fit(train_features, train_labels)
dt_predictions = dt_clf.predict(test_features)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(test_labels, dt_predictions)
print("Accuracy:", accuracy)

# You can also print a classification report for more detailed metrics
print(classification_report(test_labels, dt_predictions))




Accuracy: 0.36
              precision    recall  f1-score   support

           0       0.36      0.95      0.53        83
           1       0.25      0.03      0.05        65
           2       0.00      0.00      0.00        51
           3       0.00      0.00      0.00        26

    accuracy                           0.36       225
   macro avg       0.15      0.25      0.15       225
weighted avg       0.21      0.36      0.21       225



In [ ]:
f1_score(dt_predictions, test_labels,average='weighted')

0.5098889903602232

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=200, max_depth=3)
rf_clf.fit(train_features, train_labels)
rf_predictions = rf_clf.predict(test_features)
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(test_labels, rf_predictions)
print("Accuracy:", accuracy)

# You can also print a classification report for more detailed metrics
print(classification_report(test_labels, rf_predictions))



Accuracy: 0.36
              precision    recall  f1-score   support

           0       0.37      0.95      0.53        83
           1       0.25      0.03      0.05        65
           2       0.00      0.00      0.00        51
           3       0.00      0.00      0.00        26

    accuracy                           0.36       225
   macro avg       0.15      0.25      0.15       225
weighted avg       0.21      0.36      0.21       225



In [ ]:
f1_score(rf_predictions, test_labels,average='weighted')

0.5092392195191481

In [ ]:
from sklearn.ensemble import VotingClassifier

# Create a VotingClassifier with your individual models
voting_clf = VotingClassifier(estimators=[
    ('lr', lr_clf),  # Replace lr_clf with your trained Logistic Regression model
    ('rf', rf_clf),  # Replace rf_clf with your trained Random Forest model
    ('dt', dt_clf)   # Replace dt_clf with your trained DecisionTreeClassifier model
], voting='hard')  # 'hard' for majority vote, or 'soft' for weighted vote

# Fit the VotingClassifier on your training data (optional)
voting_clf.fit(train_features, train_labels)
ensemble_predictions = voting_clf.predict(test_features)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

ensemble_accuracy = accuracy_score(test_labels, ensemble_predictions)
ensemble_f1 = f1_score(test_labels, ensemble_predictions, average='weighted')

print("Ensemble Accuracy:", ensemble_accuracy)
print("Ensemble F1-score:", ensemble_f1)


Ensemble Accuracy: 0.3688888888888889
Ensemble F1-score: 0.20685010178934962
